In [1]:
import os
import sys
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'
notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from keras_core import utils
from aepy.data.datasets import load_MNIST
from aepy.data.utils import evaluate
from aepy.models.base.default_architectures import (Decoder_Conv_MNIST, Decoder_MLP,
                                               Encoder_Conv_MNIST, Encoder_MLP)
from aepy.models.vae.vae_model import VAE, VAECallback
from aepy.pipelines.training import TrainingPipeline

2023-12-20 00:11:45.356445: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 00:11:45.356474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 00:11:45.357257: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 00:11:45.362346: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 00:11:45.897909: W tensorflow/compiler/tf2

Using TensorFlow backend


In [2]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

print(tf.__version__)

GPU found
2.15.0


2023-12-20 00:11:47.116531: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 00:11:47.133767: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 00:11:47.133942: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

#x_train = np.expand_dims(x_train, -1).astype("float32") / 255
#x_test = np.expand_dims(x_test, -1).astype("float32") / 255

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-12-20 00:11:47 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-12-20 00:11:47 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-12-20 00:11:47 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-12-20 00:11:47 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [4]:
train_data = dict(data=x_train.astype(float), labels=y_train)
test_data = dict(data=x_test.astype(float), labels=y_test)
print(y_test.shape)
print(x_test.shape)

(10000, 10)
(10000, 28, 28, 1)


In [5]:
# Model creation
model = VAE(input_dim=(x_train.shape[1], x_train.shape[2]), latent_dim=10, downstream_task='classification',
            encoder=Encoder_Conv_MNIST, decoder=Decoder_Conv_MNIST, layers_conf=[32, 64], n_classes=n_classes,
            weight_vae=0.25, weight_clf=20)

2023-12-20 00:11:47 [INFO]: Classificator available for the latent space of the autoencoder
2023-12-20 00:11:47.446766: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 00:11:47.446950: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-20 00:11:47.447078: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more 

In [7]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=6, batch_size=128)

trained_model = pipe(x=x_train.astype(float), y=y_train)

2023-12-20 00:12:23 [INFO]: +++ training_pipeline +++
2023-12-20 00:12:23 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-12-20_00-12-23


Epoch 1/6


2023-12-20 00:12:24.328725: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


End epoch 0 of training; got log keys: ['clf_loss', 'kl_loss', 'loss', 'reconstruction_loss', 'total_loss']
469/469 - 33s - 71ms/step - clf_loss: 0.0000e+00 - kl_loss: 0.0000e+00 - loss: 120449279197184.0000 - reconstruction_loss: 0.0000e+00 - total_loss: 0.0000e+00
Epoch 2/6


KeyboardInterrupt: 

In [ ]:
model.load_weights(filepath='../output_dir/training_pipeline_2023-11-25_00-50-30/model.weights.h5')

In [ ]:
def plot_label_clusters(vae,  must be equal, but ar
    # Display a 2D plot of the digit classes in the latent space
    _, _, z = vae.predict(data)
    plt.figure(figsize=(12, 10))
    plt.scatter(z[:, 0], z[:, 1], c=labels)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.show()
    if include_flow:
      z = vae.flow.inverse(z)
      plt.figure(figsize=(12, 10))
      plt.scatter(z[:, 0], z[:, 1], c=labels)
      plt.colorbar()
      plt.xlabel("z[0]")
      plt.ylabel("z[1]")
      plt.show()

In [ ]:
plot_label_clusters(model, train_data, y_train)
plot_label_clusters(model, test_data, y_test)

In [ ]:
from sklearn.metrics import classification_report

#y_train_predict = model.predict(train_data)
y_test_predict =  model.predict(test_data)

print(np.argmax(y_test_predict['clf'], axis=1))
print(np.argmax(y_test, axis=1))

target_names = ['number 0', 'number 1', 'number 2', 'number 3', 'number 4', 'number 5', 'number 6', 'number 7', 'number 8', 'number 9']

#classification_report(y_train, np.argmax(y_train_predict['clf'], axis=1), target_names=target_names)
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_test_predict['clf'], axis=1), target_names=target_names))

In [ ]:
y_hat = trained_model.predict(test_data)

evaluate(y_true=np.argmax(test_data['labels'], axis=1), 
         y_hat=np.argmax(y_hat['clf'], axis=1),
         sel_metric=accuracy_score)